<a href="https://colab.research.google.com/github/nihalB05/Google-merchendise-store-customer-revenue-prediction-project-/blob/main/project_googlecustrev_preprocessing_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

ga_customer_revenue_prediction_path = kagglehub.competition_download('ga-customer-revenue-prediction')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ga-customer-revenue-prediction/sample_submission.csv
/kaggle/input/ga-customer-revenue-prediction/train_v2.csv
/kaggle/input/ga-customer-revenue-prediction/test_v2.csv
/kaggle/input/ga-customer-revenue-prediction/sample_submission_v2.csv
/kaggle/input/ga-customer-revenue-prediction/train.csv
/kaggle/input/ga-customer-revenue-prediction/test.csv


In [ ]:
import pandas as pd
from pandas import json_normalize
import json
from joblib import Parallel, delayed
import numpy as np
import gc
import logging

# Set up logging
logging.basicConfig(filename="chunk_errors.log", level=logging.ERROR)

# Function to fix JSON strings
def fix_json_string(value):
    if isinstance(value, str):
        return value.replace("'", '"')
    return value

# Function to parse list JSON columns
def list_json_parse(df, col):
    exploded = df[col].explode().reset_index()
    expanded = json_normalize(exploded[col])
    expanded.columns = [f"{col}_{c}" for c in expanded.columns]
    if f"{col}_index" in expanded.columns and f"{col}_value" in expanded.columns:
        pivoted = expanded.pivot(
            index="index",
            columns=f"{col}_index",
            values=f"{col}_value"
        )
        pivoted.columns = [f"{col}_{c}" for c in pivoted.columns]
    else:
        pivoted = expanded
    df = df.join(pivoted, how="left").drop(col, axis=1)
    return df

# Function to parse JSON columns
def parse_json_in_chunk(chunk):
    json_cols = ["device", "geoNetwork", "totals", "trafficSource"]
    for col in json_cols:
        if col in chunk.columns:
            chunk[col] = chunk[col].apply(json.loads)
            expanded = json_normalize(chunk[col])
            expanded.columns = [f"{col}_{c}" for c in expanded.columns]
            chunk = chunk.join(expanded).drop(col, axis=1)
    return chunk

# Function to process a single chunk
def process_chunk(chunk):
    # Step 1: Fix JSON strings
    json_columns = ["customDimensions", "hits"]
    for col in json_columns:
        if col in chunk.columns:
            chunk[col] = chunk[col].apply(fix_json_string)
            chunk = list_json_parse(chunk, col)

    # Step 2: Parse JSON columns
    chunk = parse_json_in_chunk(chunk)

    # Step 3: Replace placeholder values with NaN
    placeholder_values = [
        "not available in demo dataset",
        "(not set)",
        "",
        " : ",
        "unknown.unknown",
        "NaN",
        "null",
        "None"
    ]
    chunk.replace(placeholder_values, np.nan, inplace=True)

    return chunk

# Function to process and save a chunk
def process_and_save_chunk(i, chunk):
    try:
        processed_chunk = process_chunk(chunk)
        processed_chunk.to_csv(f"processed_chunk_{i}.csv", index=False)
        del processed_chunk
        gc.collect()
        return f"processed_chunk_{i}.csv"
    except Exception as e:
        logging.error(f"Error processing chunk {i}: {e}")
        return None

# Define chunk size
chunk_size = 10000  # Reduced chunk size

# Define file path (platform-independent)
file_path = r"/kaggle/input/ga-customer-revenue-prediction/train_v2.csv"

# Process CSV in chunks using parallel processing
processed_chunks = Parallel(n_jobs=-1, timeout=9999, pre_dispatch="2*n_jobs")(
    delayed(process_and_save_chunk)(i, chunk)
    for i, chunk in enumerate(pd.read_csv(file_path, chunksize=chunk_size, low_memory=False))
)

# Combine processed chunks (if needed)
df_train = pd.concat([pd.read_csv(f) for f in processed_chunks if f is not None], ignore_index=True)

In [ ]:
df_train.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_referralPath,trafficSource_isTrueDirect,trafficSource_adContent,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_campaignCode
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,NaN,NaN,...,NaN,/a/google.com/transportation/mtv-services/bike...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,NaN,NaN,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_train.to_csv("/kaggle/working/df_train_preprocessed.csv",index=False)